# **TP1**: Algoritmos de ML (DataSet: Marketing)

## Procesamiento

### Exploracion y limpieza de datos
---


##### Imports

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
pd.set_option('display.max_columns', 200)

##### Exploración

###### Entendimiento

In [8]:
df = pd.read_csv('./data/Marketing.csv', sep=';')

In [9]:
df.shape

(41188, 21)

In [4]:
df.head(5)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [5]:
df.dtypes

age                 int64
job                object
marital            object
education          object
default            object
housing            object
loan               object
contact            object
month              object
day_of_week        object
duration            int64
campaign            int64
pdays               int64
previous            int64
poutcome           object
emp.var.rate      float64
cons.price.idx    float64
cons.conf.idx     float64
euribor3m         float64
nr.employed       float64
y                  object
dtype: object

In [6]:
df.describe()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,41188.00000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000
mean,40.02406,258.285010,2.567593,962.475454,0.172963,0.081886,93.575664,-40.502600,3.621291,5167.035911
std,10.42125,259.279249,2.770014,186.910907,0.494901,1.570960,0.578840,4.628198,1.734447,72.251528
min,17.00000,0.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000
25%,32.00000,102.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.344000,5099.100000
50%,38.00000,180.000000,2.000000,999.000000,0.000000,1.100000,93.749000,-41.800000,4.857000,5191.000000
75%,47.00000,319.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000
max,98.00000,4918.000000,56.000000,999.000000,7.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000


###### Preparación de datos

In [ ]:
# Evaluar si los valores de contact podrian no ser significativos
print(f"contact tiene valores insignificativos superiores al 50% de la media? {(df["contact"] == "unknown").mean() > 0.5}")

    
# Evaluar si los valores de default podrian no ser significativos
print(df["default"].value_counts(normalize=True))

contact tiene valores insignificativos superiores al 50% de la media? False
default
no         0.791201
unknown    0.208726
yes        0.000073
Name: proportion, dtype: float64


## Respuestas

### **1. Identificación del Tipo de Problema**
---
Es un **problema de clasificación binaria**, ya que la variable objetivo (`y`) es categórica con dos clases: "yes" o "no". El objetivo es predecir si un cliente suscribirá un producto financiero (ejemplo típico de marketing bancario).

### **2. Variables y sus Características**  
---
El dataset contiene **21 variables**:

| Variable           | Tipo               | Descripción/Rango                                     | Ejemplo               |
|---------------------|--------------------|-------------------------------------------------------|-----------------------|
| **age**            | Numérica continua  | Edad del cliente (ej. 56, 57)                         | 56                   |
| **job**            | Categórica nominal | Ocupación (12 categorías: "admin.", "blue-collar", etc.) | "housemaid"          |
| **marital**        | Categórica nominal | Estado civil ("married", "single", "divorced")        | "married"            |
| **education**      | Categórica ordinal | Nivel educativo ("basic.4y", "high.school", etc.)     | "basic.4y"           |
| **default**        | Categórica binaria | ¿Tiene crédito en mora? ("no", "yes", "unknown")       | "no"                 |
| **housing**        | Categórica binaria | ¿Tiene préstamo hipotecario? ("no", "yes")             | "no"                 |
| **loan**           | Categórica binaria | ¿Tiene préstamo personal? ("no", "yes")                | "no"                 |
| **contact**        | Categórica nominal | Medio de contacto ("telephone", "cellular")            | "telephone"          |
| **month**          | Categórica cíclica | Mes del último contacto ("may", "jun", etc.)           | "may"                |
| **day_of_week**    | Categórica cíclica | Día de la semana ("mon", "tue", etc.)                  | "mon"                |
| **duration**       | Numérica continua  | Duración del último contacto (segundos)                | 261                  |
| **campaign**       | Numérica discreta  | Número de contactos en esta campaña                   | 1                    |
| **pdays**          | Numérica discreta  | Días desde último contacto (999 = no contactado)       | 999                  |
| **previous**       | Numérica discreta  | Número de contactos antes de esta campaña             | 0                    |
| **poutcome**       | Categórica nominal | Resultado campaña anterior ("nonexistent", "success") | "nonexistent"        |
| **emp.var.rate**   | Numérica continua  | Tasa de variación empleo (económica)                  | 1.1                  |
| **cons.price.idx** | Numérica continua  | Índice de precios al consumidor                       | 93.994               |
| **cons.conf.idx**  | Numérica continua  | Índice de confianza del consumidor                    | -36.4                |
| **euribor3m**      | Numérica continua  | Tasa Euribor a 3 meses                                | 4.857                |
| **nr.employed**    | Numérica continua  | Número de empleados (económica)                       | 5191                 |
| **y**             | Categórica binaria | ¿Suscribió el producto? ("no", "yes")                 | "no"                 |

**Observaciones clave**:  
- Variables como `pdays=999` y `poutcome="nonexistent"` indican que no hubo contacto previo.  
- `duration` puede causar **fugas de datos** (se conoce después del contacto), pero se incluirá con advertencias.  
- Variables económicas (`emp.var.rate`, `euribor3m`) pueden estar correlacionadas.